In [2]:
import pyarrow.parquet as pq
import pandas as pd
import pyarrow as pa

## Create a Parquet file

How can we create a Parquet file in Python?

Let's start from a Python DataFrame

In [2]:
df = pd.DataFrame(
    {
        'one': [-1, 0, 2.5],
        'two': ['foo', 'bar', 'baz'],
        'three': [True, False, True]
    },
    index=list('abc')
)

We then use the Apache Arrow _specification_.


> Apache Arrow was born from the need for a **set of standards** around tabular data representation and interchange between systems. The adoption of these standards reduces computing costs of data serialization/deserialization and implementation costs across systems implemented in different programming languages.

In Python, we can use PyArrow, the Python implementation of the Arrow specifications.

In [4]:

# pyarrow.Table object
"""The PyArrow Table type is not part of the Apache Arrow specification, but is rather a tool to help with wrangling multiple record batches and array pieces as a single logical dataset. As a relevant example, we may receive multiple small record batches in a socket stream, then need to concatenate them into contiguous memory for use in NumPy or pandas. The Table object makes this efficient without requiring additional memory copying."""
table = pa.Table.from_pandas(df)

# https://arrow.apache.org/docs/python/generated/pyarrow.parquet.write_table.html#pyarrow.parquet.write_table
pq.write_table(table, 'example.parquet')

Parquet metadata...

In [9]:
pq.read_metadata('example.parquet')

  created_by: parquet-cpp-arrow version 18.0.0
  num_columns: 4
  num_rows: 3
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 2572

What is metadata?
https://parquet.apache.org/docs/file-format/metadata/
We have just read the `FileMetadata`

> The file metadata is described by the `FileMetaData` structure. This file metadata provides offset and size information useful when navigating the Parquet file. 

![Parquet Metadata](docs/FileFormat.gif)

Let's take a larger file

https://www.stats.govt.nz/large-datasets/csv-files-for-download/
https://www.stats.govt.nz/assets/Uploads/New-Zealand-business-demography-statistics/New-Zealand-business-demography-statistics-At-February-2024/Download-data/geographic-units-by-industry-and-statistical-area-2000-2024-descending-order.zip

In [3]:
df_large = pd.read_csv('./geographic-units-by-industry-and-statistical-area-2000-2024-descending-order/geographic-units-by-industry-and-statistical-area-2000-2024-descending-order-february-2024.csv')
print(df_large.head())
print(df_large.shape)

  anzsic06     Area  year  geo_count  ec_count
0        A  A100100  2024         87       200
1        A  A100200  2024        135       210
2        A  A100301  2024          6        35
3        A  A100400  2024         54        35
4        A  A100500  2024         51        95
(6751326, 5)


We write again this table to a Parquet file

In [ ]:
pq.write_table(pa.Table.from_pandas(df_large), 'example_large.parquet')

Let's dig more into the Metadata file

In [10]:
parquet_file = pq.ParquetFile('example_large.parquet')
parquet_file.metadata

  created_by: parquet-cpp-arrow version 18.0.0
  num_columns: 5
  num_rows: 6751326
  num_row_groups: 6752
  format_version: 2.6
  serialized_size: 3260180

## Is Parquet 100% columnar?

What are row groups?
Ref:
https://blog.det.life/i-spent-8-hours-learning-parquet-heres-what-i-discovered-97add13fb28f

Traditional row-wise formats store data as records, one after another, much like a database table. This format is intuitive and works well when accessing entire records frequently. However, it can be inefficient when dealing with analytics, where you often only need specific columns from a large dataset.

![row storage](./docs/row-storage.webp)

Columnar formats address this issue by storing data in columns instead of rows. This means that when you need specific columns, you can read only the data you need, significantly reducing the amount of data scanned.

![columnar storage](./docs/columnar-storage.webp)

However, simply storing data in a columnar format has some downsides. The record **write or update** operation requires touching **multiple column segments**, resulting in numerous **I/O** operations. This can significantly slow the write performance, especially when dealing with large datasets.

In addition, when queries involve multiple columns, the database system must reconstruct the records from separate columns. The cost of this reconstruction increases with the **number of columns** involved in the query.

![row groups](./docs/row-groups.webp)

The format groups data into “row groups,” each containing a subset of rows. (horizontal partition.) Within each row group, data for each column is called a “column chunk.” (vertical partition)



## Page header

We will not cover a further level of granular metadata: `PageHeader`

The page header metadata is stored with the page data and includes information such as value encoding, definition encoding, and repetition encoding. In addition to the data values, Parquet also stores definition and repetition levels to handle nested data. The application uses the page header to **read and decode** the data.

In [17]:
## let's look att Row Group metadata

parquet_file.metadata.row_group(0)

  num_columns: 5
  num_rows: 1000
  total_byte_size: 15617
  sorting_columns: ()

In [16]:
parquet_file.metadata.row_group(0).column(2)

  file_offset: 0
  file_path: 
  physical_type: INT64
  num_values: 1000
  path_in_schema: year
  is_stats_set: True
  statistics:
      has_min_max: True
      min: 2024
      max: 2024
      null_count: 0
      distinct_count: None
      num_values: 1000
      physical_type: INT64
      logical_type: None
      converted_type (legacy): NONE
  compression: SNAPPY
  encodings: ('PLAIN', 'RLE', 'RLE_DICTIONARY')
  has_dictionary_page: True
  dictionary_page_offset: 5607
  data_page_offset: 5631
  total_compressed_size: 99
  total_uncompressed_size: 95

Why is this statistic useful?
Let's try a basic count

In [8]:
df_large = pd.read_csv('./geographic-units-by-industry-and-statistical-area-2000-2024-descending-order/geographic-units-by-industry-and-statistical-area-2000-2024-descending-order-february-2024.csv')
print(df_large.count())

anzsic06     6751326
Area         6751326
year         6751326
geo_count    6751326
ec_count     6751326
dtype: int64


In [7]:
parquet_file = pq.ParquetFile('example_large.parquet')
row_count = parquet_file.metadata.num_rows
print(f"Total number of rows: {row_count}")
#given this pyarrow parquet file, compute the count of rows in the year column.

Total number of rows: 6751326


Something more sophisticated, count the rows per year

In [15]:
df_large = pd.read_csv('./geographic-units-by-industry-and-statistical-area-2000-2024-descending-order/geographic-units-by-industry-and-statistical-area-2000-2024-descending-order-february-2024.csv')
count_per_year = df_large.groupby('year').size()
print(count_per_year)

year
2000    247192
2001    245753
2002    246695
2003    250130
2004    259121
2005    263223
2006    265095
2007    266518
2008    267211
2009    268547
2010    267549
2011    267954
2012    268210
2013    267911
2014    270514
2015    273210
2016    275177
2017    277515
2018    278822
2019    281313
2020    282621
2021    283621
2022    290242
2023    293312
2024    293870
dtype: int64


If we avoid using Pandas, but just PyArrow engine?

In [16]:
parquet_file = pq.ParquetFile('example_large.parquet')
dataset = ds.dataset('example_large.parquet', format='parquet', )
year_column = dataset.to_table(columns=['year'])
count_per_year = year_column.group_by('year').aggregate([('year', 'count')])
print(count_per_year)

pyarrow.Table
year: int64
year_count: int64
----
year: [[2024,2023,2022,2021,2020,...,2004,2003,2002,2001,2000]]
year_count: [[293870,293312,290242,283621,282621,...,259121,250130,246695,245753,247192]]


Have we been lucky?

In [12]:
import timeit

execution_times = []

for _ in range(10):
    start_time = timeit.default_timer()
    
    # Your code
    df_large = pd.read_csv('./geographic-units-by-industry-and-statistical-area-2000-2024-descending-order/geographic-units-by-industry-and-statistical-area-2000-2024-descending-order-february-2024.csv')
    count_per_year = df_large.groupby('year').size()
    
    end_time = timeit.default_timer()
    execution_time = end_time - start_time
    execution_times.append(execution_time)

average_execution_time = sum(execution_times) / len(execution_times)
print(f"Average execution time: {average_execution_time} seconds")

Average execution time: 0.8749954289989545 seconds


In [13]:

import pyarrow.dataset as ds

import timeit

execution_times = []

for _ in range(10):
    start_time = timeit.default_timer()
    
    # Your code
    parquet_file = pq.ParquetFile('example_large.parquet')
    dataset = ds.dataset('example_large.parquet', format='parquet', )
    year_column = dataset.to_table(columns=['year'])
    count_per_year = year_column.group_by('year').aggregate([('year', 'count')])


    end_time = timeit.default_timer()
    execution_time = end_time - start_time
    execution_times.append(execution_time)

average_execution_time = sum(execution_times) / len(execution_times)
print(f"Average execution time: {average_execution_time} seconds")


Average execution time: 0.23891639159992337 seconds


In [4]:
pq.read_metadata('example_large.parquet')

  created_by: parquet-cpp-arrow version 18.0.0
  num_columns: 5
  num_rows: 6751326
  num_row_groups: 6752
  format_version: 2.6
  serialized_size: 3260180

Ref row group
https://blog.det.life/i-spent-8-hours-learning-parquet-heres-what-i-discovered-97add13fb28f

Imagine 6M records is large, we can partition

In [8]:
distinct_years = df_large['year'].unique()
print(distinct_years)

[2024 2023 2022 2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011
 2010 2009 2008 2007 2006 2005 2004 2003 2002 2001 2000]


In [ ]:
pq.write_table(
    pa.Table.from_pandas(df_large),
    'example_large.parquet',
    ProgressBar
)